Es el momento de meter todos nuestros datos en SQL 💪🏽!!! En este ejercicio nos crearemos dos tablas en una BBDD creada por nosotras.
📌 Nota Todo lo tendremos que hacer desde jupyter notebook

1. Cread la BBDD con el nombre de energía. La BBDD tendrá la siguiente forma:

2. Cread las tablas de la BBDD:
    - Tabla fechas
    - Tabla nacional_renovable_no_renovable
    - Tabla comunidades_renovable_no_renovable
    - Tabla comunidades
    
3. BONUS Insertar los datos en las tablas. 📌 Nota Esta parte del pair es optativa y no será considerada para la evaluación

In [1]:
import pandas as pd
import mysql.connector

Cargamos datos

In [2]:
df_renovables = pd.read_pickle("datos/tipo_energia.pkl")
df_comunidades = pd.read_pickle("datos/datos_comunidades.pkl")

In [3]:
display(df_renovables.head(1))
display(df_comunidades.head(1))

,value,percentage,datetime,tipo_energia,fecha
0,86667052.85,0.31,2011-01-01T00:00:00.000+01:00,Renovable,2011-01-01


,value,percentage,datetime,tipo_energia,comunidad,cod_comunidad,fecha
0,202972.89,1.0,2011-01-01T00:00:00.000+01:00,No renovable,Ceuta,8744,2011-01-01


Creamos la base de datos

In [4]:
def create_DB(nombre_bbdd):

    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="AlumnaAdalab" # aquí tendréis que poner vuestra contraseña de MySQL
    )
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE SCHEMA IF NOT EXISTS {nombre_bbdd} DEFAULT CHARACTER SET utf8 ;")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)


In [5]:
energy_DB = create_DB("energia")

Conexión realizada con éxito
CMySQLCursor: CREATE SCHEMA IF NOT EXISTS energia DEFA..


In [6]:
def create_table(nombre_bbdd, pwd, query):
    
    # nos conectamsos con el servidor usando el conector de sql
    cnx = mysql.connector.connect(user='root', password=f"{pwd}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
    # iniciamos el cursor
    mycursor = cnx.cursor()
    
    # intentamos hacer la query
    try: 
        mycursor.execute(query)
        cnx.commit() 
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [7]:
tabla_fechas = """
CREATE TABLE IF NOT EXISTS `energia`.`Fechas` (
  `idFechas` INT NOT NULL AUTO_INCREMENT,
  `fecha` DATE NOT NULL,
  PRIMARY KEY (`idFechas`))
ENGINE = InnoDB;
"""

In [8]:
tabla_nacional_R_nR = """
CREATE TABLE IF NOT EXISTS `energia`.`nacional_renovable_no_renovable` (
  `idnacional_renovable_no_renovable` INT NOT NULL AUTO_INCREMENT,
  `porcentaje` INT NOT NULL,
  `tipo_energia` VARCHAR(45) NOT NULL,
  `valor` DECIMAL NOT NULL,
  `Fechas_idFechas` INT NOT NULL,
  PRIMARY KEY (`idnacional_renovable_no_renovable`),
  INDEX `fk_nacional_renovable_no_renovable_Fechas_idx` (`Fechas_idFechas` ASC) VISIBLE,
  CONSTRAINT `fk_nacional_renovable_no_renovable_Fechas`
    FOREIGN KEY (`Fechas_idFechas`)
    REFERENCES `energia`.`Fechas` (`idFechas`)
    ON DELETE CASCADE
    ON UPDATE CASCADE)
ENGINE = InnoDB;
"""


In [9]:
tabla_comunidades = """
CREATE TABLE IF NOT EXISTS `energia`.`comunidades` (
  `idcomunidades` INT NOT NULL AUTO_INCREMENT,
  `comunidad` VARCHAR(45) NOT NULL,
  PRIMARY KEY (`idcomunidades`))
ENGINE = InnoDB;
"""

In [10]:
tabla_comunidades_R_nR = """
CREATE TABLE IF NOT EXISTS `energia`.`comunidades_renovable_no_renovable` (
  `idcomunidades_renovable_no_renovable` INT NOT NULL AUTO_INCREMENT,
  `porcentaje` INT NOT NULL,
  `tipo_energia` VARCHAR(45) NOT NULL,
  `valor` DECIMAL NOT NULL,
  `Fechas_idFechas` INT NOT NULL,
  `comunidades_idcomunidades` INT NOT NULL,
  PRIMARY KEY (`idcomunidades_renovable_no_renovable`),
  INDEX `fk_comunidades_renovable_no_renovable_Fechas1_idx` (`Fechas_idFechas` ASC) VISIBLE,
  INDEX `fk_comunidades_renovable_no_renovable_comunidades1_idx` (`comunidades_idcomunidades` ASC) VISIBLE,
  CONSTRAINT `fk_comunidades_renovable_no_renovable_Fechas1`
    FOREIGN KEY (`Fechas_idFechas`)
    REFERENCES `energia`.`Fechas` (`idFechas`)
    ON DELETE CASCADE
    ON UPDATE CASCADE,
  CONSTRAINT `fk_comunidades_renovable_no_renovable_comunidades1`
    FOREIGN KEY (`comunidades_idcomunidades`)
    REFERENCES `energia`.`comunidades` (`idcomunidades`)
    ON DELETE CASCADE
    ON UPDATE CASCADE)
ENGINE = InnoDB;
"""

In [11]:
create_table("energia", "AlumnaAdalab", tabla_fechas)

In [12]:
create_table("energia", "AlumnaAdalab", tabla_nacional_R_nR)
create_table("energia", "AlumnaAdalab", tabla_comunidades)
create_table("energia", "AlumnaAdalab", tabla_comunidades_R_nR)

Insertamos los datos

In [13]:
def insert_into(nombre_bbdd, pwd, query):
    
    # nos conectamsos con el servidor usando el conector de sql
    cnx = mysql.connector.connect(user='root', password=f"{pwd}",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
    # iniciamos el cursor
    mycursor = cnx.cursor()
    
    # intentamos hacer la query
    try: 
        mycursor.execute(query)
        cnx.commit() 
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [15]:
for indice, fila in df_renovables.iterrows(): 
    fila["fecha"] = str(fila["fecha"]).split(" ")[0]
    query_fechas = f"""
            INSERT INTO Fechas (fecha) 
            VALUES ("{fila['fecha']}");
            """
    
    insert_into("energia", "AlumnaAdalab", query_fechas)

Comprobamos si existen las fechas

In [16]:
def check_fechas(pwd, nombre_bbdd):
    
    cnx = mysql.connector.connect(user='root', password=f'{pwd}',
                                  host='127.0.0.1', database=f"{nombre_bbdd}")
    mycursor = cnx.cursor()


    query_existe_fecha = f"""
            SELECT DISTINCT fecha FROM Fechas
            """
    mycursor.execute(query_existe_fecha)
    dates = mycursor.fetchall()
    return dates

In [17]:
check_fechas("AlumnaAdalab", "energia")[0]

(datetime.date(2011, 1, 1),)

In [18]:
df_renovables.head(1)

,value,percentage,datetime,tipo_energia,fecha
0,86667052.85,0.31,2011-01-01T00:00:00.000+01:00,Renovable,2011-01-01
